In [82]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
from proj1_helpers import *
from implementations import *

In [84]:
def standardize(x, mean_x=None, std_x=None):
    """Standardize the original data set."""
    if mean_x is None:
        mean_x = np.nanmean(x, axis=0)
    if std_x is None:
        std_x = np.nanstd(x, axis=0)
    x = x - mean_x
    x = x / std_x
    return x, mean_x, std_x

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

def split_data(x, y, ratio, myseed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(myseed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [85]:
def accuracy(y_pred, y_true):
    return (y_pred == y_true).sum()/len(y_true)

In [86]:
# data path
train_data_path = "train.csv"
test_data_path = "test.csv"

# load training and test set
y_train_data, x_train_data, id_train_data = load_csv_data(train_data_path)
y_test_data, x_test_data, id_test_data = load_csv_data(test_data_path)

In [87]:
long_tail = [0, 1, 2, 3, 5, 8, 9, 10, 13, 16, 19, 21, 23, 26, 29]

In [88]:
jet0_drop = [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29]
jet1_drop = [4, 5, 6, 12, 22, 26, 27, 28]
jet2_drop = [22]
jet3_drop = [22]

In [89]:
jet0_index = np.where(x_train_data[:,22]==0)[0]
jet1_index = np.where(x_train_data[:,22]==1)[0]
jet2_index = np.where(x_train_data[:,22]==2)[0]
jet3_index = np.where(x_train_data[:,22]==3)[0]

In [90]:
x_train_data[x_train_data==-999] = np.nan

In [91]:
x_train_data[:, long_tail] = np.log1p(x_train_data[:, long_tail])

D:\python\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log1p
  """Entry point for launching an IPython kernel.


In [92]:
mean = np.nanmean(x_train_data, 0)
std = np.nanstd(x_train_data, 0)

In [93]:
x_train_data -= mean
x_train_data /= std

In [94]:
x_train_data = np.nan_to_num(x_train_data)

In [96]:
x_jet0 = np.delete(x_train_data[jet0_index, :], jet0_drop, axis=1)
x_jet1 = np.delete(x_train_data[jet1_index, :], jet1_drop, axis=1)
x_jet2 = np.delete(x_train_data[jet2_index, :], jet2_drop, axis=1)
x_jet3 = np.delete(x_train_data[jet3_index, :], jet3_drop, axis=1)

In [97]:
x_jet0.shape, x_jet1.shape, x_jet2.shape, x_jet3.shape

((99913, 18), (77544, 22), (50379, 29), (22164, 29))

In [98]:
y_jet0 = y_train_data[jet0_index]
y_jet1 = y_train_data[jet1_index]
y_jet2 = y_train_data[jet2_index]
y_jet3 = y_train_data[jet3_index]

In [99]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

In [100]:
# normalizatio after build ploy might be needed!
tx_jet0 = build_poly(x_jet0, 15)
tx_jet1 = build_poly(x_jet1, 15)
tx_jet2 = build_poly(x_jet2, 15)
tx_jet3 = build_poly(x_jet3, 15)

# Least Squares GD

In [183]:
tx_jet0 = build_poly(x_jet0, 10)
tx_jet1 = build_poly(x_jet1, 10)
tx_jet2 = build_poly(x_jet2, 10)
tx_jet3 = build_poly(x_jet3, 10)
gamma=0.01
max_iters=100
initial_w_jet0=np.zeros((tx_jet0.shape[1], 1))
initial_w_jet1=np.zeros((tx_jet1.shape[1], 1))
initial_w_jet2=np.zeros((tx_jet2.shape[1], 1))
initial_w_jet3=np.zeros((tx_jet3.shape[1], 1))
w_jet0, _ = least_squares_GD(y_jet0, tx_jet0, initial_w_jet0, max_iters, gamma)
w_jet1, _ = least_squares_GD(y_jet1, tx_jet1, initial_w_jet1, max_iters, gamma)
w_jet2, _ = least_squares_GD(y_jet2, tx_jet2, initial_w_jet2, max_iters, gamma)
w_jet3, _ = least_squares_GD(y_jet3, tx_jet3, initial_w_jet3, max_iters, gamma)

MemoryError: 

In [ ]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3)

In [ ]:
accuracy(y_pred, y_train_data)

# Least Squares SGD

In [184]:
tx_jet0 = build_poly(x_jet0, 1)
tx_jet1 = build_poly(x_jet1, 1)
tx_jet2 = build_poly(x_jet2, 1)
tx_jet3 = build_poly(x_jet3, 1)
gamma=0.01
max_iters=10
initial_w_jet0=np.zeros((tx_jet0.shape[1], 1))
initial_w_jet1=np.zeros((tx_jet1.shape[1], 1))
initial_w_jet2=np.zeros((tx_jet2.shape[1], 1))
initial_w_jet3=np.zeros((tx_jet3.shape[1], 1))
w_jet0, _ = least_squares_SGD(y_jet0, tx_jet0, initial_w_jet0, max_iters, gamma)
w_jet1, _ = least_squares_SGD(y_jet1, tx_jet1, initial_w_jet1, max_iters, gamma)
w_jet2, _ = least_squares_SGD(y_jet2, tx_jet2, initial_w_jet2, max_iters, gamma)
w_jet3, _ = least_squares_SGD(y_jet3, tx_jet3, initial_w_jet3, max_iters, gamma)

MemoryError: 

In [ ]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3)
accuracy(y_pred, y_train_data)

# Least Squares

In [179]:
tx_jet0 = build_poly(x_jet0, 15)
tx_jet1 = build_poly(x_jet1, 15)
tx_jet2 = build_poly(x_jet2, 15)
tx_jet3 = build_poly(x_jet3, 15)
w_jet0, _ = least_squares(y_jet0, tx_jet0)
w_jet1, _ = least_squares(y_jet1, tx_jet1)
w_jet2, _ = least_squares(y_jet2, tx_jet2)
w_jet3, _ = least_squares(y_jet3, tx_jet3)

In [180]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3)
accuracy(y_pred, y_train_data)

0.832992

## Ridge Regression

In [150]:
tx_jet0 = build_poly(x_jet0, 15)
tx_jet1 = build_poly(x_jet1, 15)
tx_jet2 = build_poly(x_jet2, 15)
tx_jet3 = build_poly(x_jet3, 15)

In [151]:
lambda_ = 0.001
w_jet0, _ = ridge_regression(y_jet0, tx_jet0, lambda_)
w_jet1, _ = ridge_regression(y_jet1, tx_jet1, lambda_)
w_jet2, _ = ridge_regression(y_jet2, tx_jet2, lambda_)
w_jet3, _ = ridge_regression(y_jet3, tx_jet3, lambda_)

In [152]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3)

In [153]:
accuracy(y_pred, y_train_data)

0.832848

# Logsitic Regression

In [171]:
tx_jet0 = build_poly(x_jet0, 1)
tx_jet1 = build_poly(x_jet1, 1)
tx_jet2 = build_poly(x_jet2, 1)
tx_jet3 = build_poly(x_jet3, 1)
gamma=0.01
max_iters=100
initial_w_jet0=np.zeros((tx_jet0.shape[1], 1))
initial_w_jet1=np.zeros((tx_jet1.shape[1], 1))
initial_w_jet2=np.zeros((tx_jet2.shape[1], 1))
initial_w_jet3=np.zeros((tx_jet3.shape[1], 1))
w_jet0, _ = logistic_regression(y_jet0, tx_jet0, initial_w_jet0, max_iters, gamma)
w_jet1, _ = logistic_regression(y_jet1, tx_jet1, initial_w_jet1, max_iters, gamma)
w_jet2, _ = logistic_regression(y_jet2, tx_jet2, initial_w_jet2, max_iters, gamma)
w_jet3, _ = logistic_regression(y_jet3, tx_jet3, initial_w_jet3, max_iters, gamma)

In [172]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0).reshape(y_pred[jet0_index].shape)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1).reshape(y_pred[jet1_index].shape)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2).reshape(y_pred[jet2_index].shape)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3).reshape(y_pred[jet3_index].shape)
accuracy(y_pred, y_train_data)

0.699192

# Regularized Logistic Regression

In [167]:
tx_jet0 = build_poly(x_jet0, 1)
tx_jet1 = build_poly(x_jet1, 1)
tx_jet2 = build_poly(x_jet2, 1)
tx_jet3 = build_poly(x_jet3, 1)
lambda_ = 0.001
gamma=0.01
max_iters=100
initial_w_jet0=np.zeros((tx_jet0.shape[1], 1))
initial_w_jet1=np.zeros((tx_jet1.shape[1], 1))
initial_w_jet2=np.zeros((tx_jet2.shape[1], 1))
initial_w_jet3=np.zeros((tx_jet3.shape[1], 1))
w_jet0, _ = reg_logistic_regression(y_jet0, tx_jet0, lambda_, initial_w_jet0, max_iters, gamma)
w_jet1, _ = reg_logistic_regression(y_jet1, tx_jet1, lambda_, initial_w_jet1, max_iters, gamma)
w_jet2, _ = reg_logistic_regression(y_jet2, tx_jet2, lambda_, initial_w_jet2, max_iters, gamma)
w_jet3, _ = reg_logistic_regression(y_jet3, tx_jet3, lambda_, initial_w_jet3, max_iters, gamma)

In [168]:
y_pred = np.zeros_like(y_train_data)
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0).reshape(y_pred[jet0_index].shape)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1).reshape(y_pred[jet1_index].shape)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2).reshape(y_pred[jet2_index].shape)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3).reshape(y_pred[jet3_index].shape)
accuracy(y_pred, y_train_data)

0.710252

## Predict with Ridge Regression 

In [40]:
jet0_index = np.where(x_test_data[:,22]==0)[0]
jet1_index = np.where(x_test_data[:,22]==1)[0]
jet2_index = np.where(x_test_data[:,22]==2)[0]
jet3_index = np.where(x_test_data[:,22]==3)[0]

x_test_data[x_test_data==-999] = np.nan

x_test_data[:, long_tail] = np.log1p(x_test_data[:, long_tail])

x_test_data -= mean
x_test_data /= std

x_test_data = np.nan_to_num(x_test_data)

x_jet0 = np.delete(x_test_data[jet0_index, :], jet0_drop, axis=1)
x_jet1 = np.delete(x_test_data[jet1_index, :], jet1_drop, axis=1)
x_jet2 = np.delete(x_test_data[jet2_index, :], jet2_drop, axis=1)
x_jet3 = np.delete(x_test_data[jet3_index, :], jet3_drop, axis=1)

tx_jet0 = build_poly(x_jet0, 15)
tx_jet1 = build_poly(x_jet1, 15)
tx_jet2 = build_poly(x_jet2, 15)
tx_jet3 = build_poly(x_jet3, 15)
w_jet0, _ = ridge_regression(y_jet0, tx_jet0, lambda_)
w_jet1, _ = ridge_regression(y_jet1, tx_jet1, lambda_)
w_jet2, _ = ridge_regression(y_jet2, tx_jet2, lambda_)
w_jet3, _ = ridge_regression(y_jet3, tx_jet3, lambda_)
y_pred = np.zeros(len(x_test_data))
y_pred[jet0_index] = predict_labels(w_jet0, tx_jet0)
y_pred[jet1_index] = predict_labels(w_jet1, tx_jet1)
y_pred[jet2_index] = predict_labels(w_jet2, tx_jet2)
y_pred[jet3_index] = predict_labels(w_jet3, tx_jet3)

D:\python\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log1p
  


In [41]:
create_csv_submission(id_test_data, y_pred, "OCT18")